# Cohort construction from the MIMIC-IV database

The objective of this notebook it to construct a cohort of adult patients from the MIMIC-IV database in order to describe the pathways of patients being ventilated.

This cohort selects patients being mechanically ventilated and collect the delivered cares before the first ventilation. 
The visits for which a patient has been ventilated the first or the second day are discarded.


## Exploratory analysis

Let start by installing and loading the required modules for this notebook. 
We use two libraries:
- `psycopg` to handle Postgresql queries (note that under Linux, this requires to install `libpq-dev`)
- `pandas` to manage the datasets

In [ ]:
# installation of the required packages
!pip3 install pandas
!pip3 install psycopg2
!pip3 install seaborn

In [ ]:
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
import pickle
from datetime import datetime 

We now open a connection to the database ... Do not forget to close the connection when you finished to play with this notebook.

In [ ]:
# Specify the right informations to connect to the database
pguser="mimicuser"
pgpassword="mimic"
dbname="mimic"

In [ ]:
try:
    conn = psycopg2.connect(f'dbname={dbname} user={pguser} password={pgpassword}')
    cur = conn.cursor()
    cur.execute("SET search_path TO mimiciv")
except (Exception, psycopg2.Error) as error:
    print("Error while opening connection to the database from PostgreSQL", error)

## Mechanical Ventilation Events

Let start by finding ICD procedure codes related to ventilation

In [ ]:
query = """SELECT * FROM d_icd_procedures WHERE long_title LIKE '%ventilation%'"""

res=sqlio.read_sql_query(query, conn)
print(res)

We have 4 codes related to mechanical ventilation ... at this stage, I'll take all of them to identify my event index. Then we create a new query to identify them ... note that the idea is to modify the later the definition of the index event easily.

In [ ]:
query_index_eventtype = """SELECT icd_code FROM d_icd_procedures WHERE long_title LIKE '%ventilation%'"""

So now ... We evaluate the number of such events! At the same time, we also count the "visits" having such events, and also the patients! 

In [ ]:
query = f"""SELECT count(*), count(DISTINCT hadm_id), count(DISTINCT subject_id) FROM procedures_icd WHERE icd_code in ({query_index_eventtype})"""
res=sqlio.read_sql_query(query, conn)
print('Number of ventilation events: %d'%(res['count'].iloc[0,0]))
print('Number of admissions with ventilation event: %d'%(res.iloc[0,1]))
print('Number of patients with ventilation event: %d'%(res.iloc[0,1]))

We see that are may have several ventilation events during the same visit ... this means that we have to take care of the definition of our event index: it has to be, for instance, the first occurrence of the event of interest during some adminission.
Here .. it seems that each patient being ventilated had only one admission.

## Dataset construction

We first look for the occurrences of our events of interest (ventilation) with two restrictions:
* we only consider ventilations that started the day after the hospitalisation (not the same date)
* we only consider the first ventilation procedure in case there are several ventilations in the same admission. This criteria is applied out of the SQL query.
* we select only adults

In [ ]:
# subquery for admissions ... I take all of them to make it simple
min_nb_days=2
query_index_events = f"""SELECT DISTINCT Admissions.admittime::Date, procevent.hadm_id, procevent.chartdate AS IndexDate FROM procedures_icd AS procevent 
                            JOIN Admissions ON Admissions.hadm_id=procevent.hadm_id
                            JOIN Patients ON Patients.subject_id=Admissions.subject_id
                            WHERE Patients.anchor_year>=18 AND procevent.icd_code IN ({query_index_eventtype}) AND Admissions.admittime+INTERVAL '{min_nb_days} days'<=procevent.chartdate"""
index_events=sqlio.read_sql_query(query_index_events, conn)
#select only the first event if multiple events for the same admission
first_index_events=index_events.groupby("hadm_id").min().reset_index()
print(first_index_events)

In [ ]:
#now .. the query to collect events (based on the admission of the query above)
query = f"""SELECT procevent.hadm_id, procedures_icd.icd_code, procevent.IndexDate, procevent.IndexDate-procedures_icd.chartdate AS delay FROM procedures_icd 
                                            JOIN ({query_index_events}) AS procevent ON procevent.hadm_id=procedures_icd.hadm_id
                                            WHERE procedures_icd.chartdate<procevent.IndexDate"""
procedures=sqlio.read_sql_query(query, conn)
#select only the events for the first occurrence of the index event
procedures=pd.merge(first_index_events, procedures, how='inner', on=['hadm_id','indexdate'])
print(procedures)

In [ ]:
# the same with drugs (we use the ndc National Drug Code)
query = f"""SELECT procevent.hadm_id, prescriptions.ndc, procevent.IndexDate, prescriptions.starttime AS start_date, prescriptions.stoptime as stop_date FROM prescriptions
                                            JOIN ({query_index_events}) AS procevent ON procevent.hadm_id=prescriptions.hadm_id
                                            WHERE prescriptions.starttime<procevent.IndexDate"""
res=sqlio.read_sql_query(query, conn)
#select only the events for the first occurrence of the index event
res=pd.merge(first_index_events, res, how='inner', on=['hadm_id','indexdate'])

#remove the 0 codes that are too difficult to use
res=res[res.ndc!="0"]
print(res)

In [ ]:
#We remove the records with zero codes that are meaningless
res=res.dropna()
#res['ndc']=res['ndc'].astype(int)
res=res.loc[res.ndc!=0]

In [ ]:
#stop_date can not be later than indexdate, so
res['stop_date']=res[['stop_date','indexdate']].min(axis=1)

Now we have to transform the data for which we have exposure periods ... into a collection of dates (it is easier to do with dates)

**Warning** this step can take time ...

In [ ]:
res_nona=res
res_nona['start_date']=res_nona['start_date'].dt.date
res_nona['stop_date']=res_nona['stop_date'].dt.date
spread_drugs=pd.concat([pd.DataFrame({'date': pd.date_range(row['start_date'], row['stop_date']),
               'hadm_id': row.hadm_id,
               'ndc': row.ndc,
               'indexdate' : row.indexdate,
               'admittime' : row.admittime}, columns=['date', 'hadm_id', 'ndc', 'indexdate','admittime']) 
           for i, row in res_nona.iterrows()], ignore_index=True)
spread_drugs['delay']=spread_drugs['indexdate']-spread_drugs['date']
spread_drugs=spread_drugs[["hadm_id",'admittime','indexdate',"ndc","delay"]]
spread_drugs

We now have two tables with the same structure (procedures and drugs) ... the structure of the database is not fully suitable for our needs

In [ ]:
#Filtering on the 100 most used procedures
freq_proc=procedures['icd_code'].value_counts().reset_index().iloc[0:100]['index'].tolist()
procedures = procedures[ procedures['icd_code'].isin(freq_proc)]
#Filtering on the 100 most used drugs
freq_drugs=spread_drugs['ndc'].value_counts().reset_index().iloc[0:100]['index'].tolist()
spread_drugs = spread_drugs[ spread_drugs['ndc'].isin(freq_drugs)]

In [ ]:
procedures

In [ ]:
#spread_drugs['delay']=spread_drugs['delay'].dt.days
#procedures['delay']=procedures['delay'].dt.days
#procedures.icd_code = procedures.icd_code.astype(int)
procedures.icd_code = "p_" + procedures.icd_code
spread_drugs.ndc = "d_" + spread_drugs.ndc
cc_mat=pd.concat([spread_drugs.rename(columns={'ndc':'code'}), procedures.rename(columns={'icd_code':'code'})])
cc_mat['delay']=cc_mat['delay'].dt.days
cc_mat['value']=np.ones(len(cc_mat))

In [ ]:
#example of a processing to prepare the matrix of a patient (to be apply to all patients in next cells)
"""
x=cc_mat[cc_mat.hadm_id==29988601]
#x['value']=np.ones(len(x))
los=(x.iloc[0]['indexdate'].to_pydatetime().date()-x.iloc[0]['admittime']).days
new_index=pd.MultiIndex.from_product([cc_mat.code.unique(), np.arange(0,los)], names=['code', 'delay'])
x_rei=x[['code', 'delay','value']].drop_duplicates().set_index(['code','delay'])
x_rei=x_rei.reindex( new_index,fill_value=0)
x_rei
"""

In [ ]:
def genMatrix(x):
    los=(x.iloc[0]['indexdate'].to_pydatetime().date()-x.iloc[0]['admittime']).days
    new_index=pd.MultiIndex.from_product([cc_mat.code.unique(), np.arange(0,los)], names=['code', 'delay'])
    x_rei=x[['code', 'delay','value']].drop_duplicates().set_index(['code','delay'])
    x_rei=x_rei.reindex( new_index,fill_value=0)
    x_rei=x_rei.reset_index()
    mat=pd.pivot_table(x_rei, columns='code', index=['delay'], values='value', aggfunc=np.sum)
    return(mat)

In [ ]:
#takes few seconds to create a dataframe that reorganize the data and complete 
mat=cc_mat.groupby('hadm_id').apply(genMatrix).fillna(0)

In [ ]:
#transformation as a list of numpy arrays
mat=mat.reset_index()
mat=mat.iloc[:,mat.columns!='delay'].set_index('hadm_id', drop=True)
data=[mat.loc[i,mat.columns!='hadm_id'].to_numpy() for i in mat.index.unique()]

In [ ]:
params={}
params['source']="mimic"        # this indicates the source of the dataset (its name, version of the generator, etc.)
params['comments']="""Care pathways of patients being ventilated in ICU."""
params['generator']="CohortConstruction.ipynb"
params['date']=datetime.now()  # keep the date at which it has been generated
# matrix dimensions
params['K']=len(data)                      
params['N']=data[0].shape[1]
params['T']=None #irregular lengths

#transpose the temporal and drug dimensions
data = [ d.transpose() for d in data]

with open('mimic_data.pkl', 'wb') as f:
    pickle.dump([None, None, data, params], f)

### Collect the labels

* 100 first columns are drugs (prefixed with "d_")
* 100 last columns are procedures (prefixed with "p_")

In [ ]:
query = f"""SELECT DISTINCT drug, ndc FROM prescriptions"""
res=sqlio.read_sql_query(query, conn)
res=res[res['ndc'].isin([str(v)[2:] for v in mat.columns.tolist()])]
drugs_labels=res.groupby('ndc').first()['drug'].tolist()

In [ ]:
query = f"""SELECT icd_code, long_title FROM d_icd_procedures"""
res=sqlio.read_sql_query(query, conn)
proc_labels=res[res['icd_code'].isin([str(v)[2:] for v in mat.columns.tolist()])]['long_title'].tolist()

In [ ]:
labels=drugs_labels+proc_labels
if len(labels) != len(mat.columns):
    print("There is an error in the labels")

In [ ]:
with open('mimic_data_labels.pkl', 'wb') as f:
    pickle.dump(labels, f)

### Collect the diagnosis

There are several diagnosis for each visit. We limit the number of diagnosis to 5 per visit. 

The diasgnosis are ordered by "importance" (see https://mimic.mit.edu/docs/iv/modules/hosp/diagnoses_icd/). 
1 is the most important.

In [ ]:
query = f"""SELECT procevent.hadm_id, d.icd_code, seq_num as importance, long_title AS label FROM diagnoses_icd as d
                                            JOIN d_icd_diagnoses AS icd ON icd.icd_code=d.icd_code
                                            JOIN ({query_index_events}) AS procevent ON procevent.hadm_id=d.hadm_id WHERE seq_num<=5 ORDER BY procevent.hadm_id, seq_num"""
res=sqlio.read_sql_query(query, conn)
with open('mimic_diagnosis.pkl', 'wb') as f:
    pickle.dump(res, f)

## Connection closure

Close the connection to the database

In [ ]:
# closing database connection.
if conn:
    conn.close()